In [2]:
import pymongo
import polars as pl

import tabulate

from dltf.sloth.sloth import sloth
from dltf.sloth.utils import parse_table
from dltf.utils.tables import table_to_tokens, is_valid_table
from dltf.utils.misc import lowercase_translator, whitespace_translator
from dltf.utils.settings import DefaultPath as dp
from dltf.utils.datalake import MongoDBDataLakeHandler

In [3]:
dlhargs = [
    'mongodb',
    'latsnaptab',
    ['sloth.latest_snapshot_tables']
]

dlh = MongoDBDataLakeHandler(*dlhargs)

In [4]:
data = pl.read_csv(f'{dp.data_path.tests}/latsnaptab/latsnaptab/results/extracted/final_results_k20_q100.csv').filter(pl.col('mode') == 'bag')
data

query_id,result_id,algorithm,mode,sloth_overlap,algorithm_overlap,set_size_q,set_size_r,set_overlap,set_union_size,bag_size_q,bag_size_r,bag_overlap,jaccard_sim,multi_jaccard_sim,containment,overlap_set_sim,proximity,area_ratio,sloth_time(s),set_time(s),bag_time(s)
i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1070758,97645,"""josie""","""bag""",21,21,24,573,21,576,84,609,21,0.036458,0.035176,0.875,0.25,1.0,0.25,0.001,0.0,0.00001
189421,449467,"""josie""","""bag""",8,8,28,1110,6,1132,44,6596,8,0.0053,0.005272,0.214286,0.136364,1.0,0.181818,0.01,0.00001,0.00001
189421,460512,"""josie""","""bag""",8,8,28,371,4,395,44,996,8,0.010127,0.010025,0.142857,0.090909,1.0,0.181818,0.001,0.0,0.0
834709,1129943,"""josie""","""bag""",8,15,24,27,13,38,32,32,15,0.342105,0.254902,0.5416667,0.40625,0.533333,0.25,0.001,0.0,0.0
570689,1097784,"""josie""","""bag""",2,2,16,107,2,121,22,230,2,0.016529,0.01626,0.125,0.090909,1.0,0.090909,0.001,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1484462,513714,"""josie""","""bag""",22,40,72,260,15,317,108,459,40,0.047319,0.045181,0.208333,0.1388889,0.55,0.2037037,0.001,0.0,0.00001
1714748,1105039,"""josie""","""bag""",6,6,17,67,2,82,24,391,6,0.02439,0.02381,0.117647,0.083333,1.0,0.25,0.001,0.0,0.0
376146,906397,"""josie""","""bag""",1,1,58,114,1,171,60,204,1,0.005848,0.005814,0.017241,0.016667,1.0,0.016667,0.001,0.00001,0.00001


In [45]:
qgroups = sorted([(q[0], g.sort(by='sloth_overlap', descending=True)) for q, g in data.group_by(by='query_id')])

In [46]:
len(qgroups)

30

In [47]:
i = 4
doc = dlh.get_table_by_numeric_id(qgroups[i][0])
print(f'i: {i}, table ID: {qgroups[i][0]}, #matches={qgroups[i][1].shape[0]}, #header_rows={len(doc["headers"])}, #rows={len(doc["content"])}, #columns={len(doc["content"][0])}, valid_columns={doc["valid_columns"]}')

i: 4, table ID: 463713, #matches=20, #header_rows=1, #rows=13, #columns=4, valid_columns=[0, 1, 1, 1]


In [48]:
qgroups[i][1].head()

query_id,result_id,algorithm,mode,sloth_overlap,algorithm_overlap,set_size_q,set_size_r,set_overlap,set_union_size,bag_size_q,bag_size_r,bag_overlap,jaccard_sim,multi_jaccard_sim,containment,overlap_set_sim,proximity,area_ratio,sloth_time(s),set_time(s),bag_time(s)
i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
463713,600257,"""josie""","""bag""",8,8,38,294,7,325,39,360,8,0.021538,0.021084,0.184211,0.179487,1.0,0.205128,0.001,0.0,0.0
463713,860312,"""josie""","""bag""",8,8,38,329,7,360,39,394,8,0.019444,0.019074,0.184211,0.179487,1.0,0.205128,0.001,0.00001,0.00001
463713,623595,"""josie""","""bag""",7,7,38,82,6,114,39,87,7,0.052632,0.05,0.157895,0.153846,1.0,0.179487,0.0,0.0,0.0
463713,829224,"""josie""","""bag""",7,7,38,462,6,494,39,556,7,0.012146,0.012,0.157895,0.153846,1.0,0.179487,0.001,0.00001,0.0
463713,1696824,"""josie""","""bag""",7,8,38,543,7,574,39,618,8,0.012195,0.012048,0.184211,0.179487,0.875,0.179487,0.001,0.0,0.0


In [49]:
pl.DataFrame(doc['content'], orient='row')

column_0,column_1,column_2,column_3
str,str,str,str
"""1999–2000""","""Barmy Aunt Boomerang""","""Sebastian Simpkins""","""20 episodes"""
"""2002""","""Taggart""","""Christie""","""Episode: ""Watertight"""""
"""2009""","""Hope Springs""","""Dean McKenzie""","""8 episodes"""
"""2010""","""Worried About The Boy""","""Kirk Brandon""","""Television film"""
"""2011""","""Sirens""","""Ashley Greenwick""","""6 episodes"""
…,…,…,…
"""2015""","""Lady Chatterley's Lover""","""Oliver Mellors""","""Television film"""
"""2016""","""Medici: Masters of Florence""","""Cosimo de' Medici""","""8 episodes; also executive pro…"
"""2017""","""Oasis""","""Peter Leigh""","""Pilot"""


In [50]:
bag_q = set(table_to_tokens(doc['content'], doc['valid_columns'], 'bag', None, set(), [lowercase_translator, whitespace_translator]))
docr = dlh.get_table_by_numeric_id(qgroups[i][1][0]['result_id'][0])
bag_r = set(table_to_tokens(docr['content'], docr['valid_columns'], 'bag', None, set(), [lowercase_translator, whitespace_translator]))
doc['_id_numeric'], len(bag_q), len(bag_r), len(bag_q | bag_r), len(bag_q & bag_r), bag_q & bag_r

(463713,
 39,
 360,
 391,
 8,
 {'2 episodes@#1',
  '20 episodes@#1',
  '3 episodes@#1',
  '6 episodes@#1',
  '8 episodes@#1',
  'Pilot@#1',
  'Television film@#1',
  'Television film@#2'})

In [51]:
IDs = {
    # ricerca di tabelle con contenuto identiche/simili
    'contenuto_simile': [1749224, 1484462, 1714748, 551590, 1324441, 1121158, 1216805, 1370219, 1216034, 1749224, 1623113, 1591122, 463713, 376146],
    
    # esempio di pulizia per tabelle sporche? Con token e robe strane in mezzo, tipo "{{nom}}"
    'pulizia_tabelle_sporche': [1249360, 1630678, 612879],
    
    # esempio di ricerca di tabelle con stessi nomi per verifica di coerenza dati
    'verifica_coerenza': [1878225, 1878225, 1598208, 766881, 1195130, 834709, 323280],

    'scartate': [570689, 1394327, 189421, 1347854, 1536126, 737775, 1070758, 461074]
}

### How many tables do we have with the previous tables and the relative result IDs?

In [52]:
query_ids = {qid for k, qlist in IDs.items() if k != 'scartate' for qid in qlist}
len(query_ids)

22

In [53]:
[q for q in qgroups[0][1]['result_id']]

[125546,
 449467,
 460512,
 22598,
 1812144,
 482507,
 728872,
 637569,
 2077501,
 510150,
 852692,
 1764104,
 609535,
 2036635,
 1870104,
 1563519,
 211830,
 12071,
 1073418,
 2030269]

In [54]:
ids = {i for qid, qgroup in qgroups if qid in query_ids and qgroup.shape[0] >= 5 for i in qgroup['result_id']}
len(ids | query_ids)

450

In [55]:
ids = ids | query_ids

### Get random tables to complete the datalake

In [56]:
i = 0
n = 0
for doc in dlh.scan_tables():
    i += 1
    if len(doc['headers']) == 1:
        n += 1
    if i == 10000:
        break
n

4287

In [57]:
import random

datalake_size = 1_000
datalake_ids = ids
n_tables = dlh.count_tables()


for doc in dlh.scan_tables():
    if doc['_id_numeric'] not in datalake_ids and is_valid_table(doc['content'], doc['valid_columns']) and len(doc['headers']) == 1 and random.random() < 0.5:
        datalake_ids.add(doc['_id_numeric'])
    if len(datalake_ids) >= datalake_size:
        break
    
    print(f'{doc["_id_numeric"]}/{n_tables}, found {len(datalake_ids)}/{datalake_size} tables', end='\r')

In [58]:
import json

IDs = {
    'all_ids': list(datalake_ids),
    'query_ids': list(query_ids)
}

with open('datalake_ids.json', 'w') as fw:
    json.dump(IDs, fw, indent=4)

IDs

{'all_ids': [4098,
  10,
  4111,
  20,
  26,
  1761308,
  31,
  4128,
  4133,
  1290283,
  43,
  4141,
  880689,
  4145,
  51,
  4155,
  4156,
  49216,
  4167,
  74,
  4171,
  4174,
  1249360,
  614481,
  80,
  83,
  84,
  4185,
  4188,
  909405,
  471135,
  4193,
  98,
  1769579,
  108,
  113,
  114,
  115,
  8313,
  121,
  1695867,
  4218,
  125,
  684160,
  1532033,
  200834,
  1216646,
  137,
  138,
  4237,
  4238,
  4239,
  144,
  4241,
  4243,
  925846,
  150,
  860312,
  4248,
  155,
  4253,
  4254,
  160,
  4257,
  162,
  4262,
  1245360,
  606400,
  4289,
  194,
  4290,
  1634500,
  4291,
  4292,
  202,
  4298,
  205,
  528592,
  4306,
  4308,
  214,
  4310,
  4325,
  4326,
  1749223,
  1749224,
  4327,
  4331,
  1593580,
  236,
  1921265,
  241,
  242,
  4340,
  245,
  4341,
  4345,
  257,
  1564930,
  1749255,
  373000,
  264,
  823562,
  1888522,
  265,
  266,
  4359,
  4360,
  4365,
  4370,
  276,
  4377,
  4379,
  1528093,
  1605917,
  285,
  4381,
  4387,
  1216805,
  29

### Get all the documents and save them

In [59]:
docs = []

mongoclient = pymongo.MongoClient()
collection = mongoclient.sloth.latest_snapshot_tables

for id in datalake_ids:
   doc = collection.find_one({f'_id_numeric': id})
   docs.append(doc) 

In [60]:
len(docs)

1000

In [61]:
with open('demo_datalake.json', 'w') as fw:
    json.dump(docs, fw)

## Inspect the tables

In [63]:
mongoclient = pymongo.MongoClient()
collection = mongoclient.sloth.latest_snapshot_tables

In [64]:
i = 0

qid = qgroups[i][0]
results = [row for row in qgroups[i][1].select('result_id', 'sloth_overlap').rows()]

In [65]:
results

[(125546, 9),
 (449467, 8),
 (460512, 8),
 (22598, 8),
 (1812144, 8),
 (482507, 8),
 (728872, 8),
 (637569, 8),
 (2077501, 8),
 (510150, 8),
 (852692, 7),
 (1764104, 7),
 (609535, 7),
 (2036635, 7),
 (1870104, 7),
 (1563519, 7),
 (211830, 7),
 (12071, 7),
 (1073418, 5),
 (2030269, 5)]

In [78]:
qdoc = collection.find_one({'_id_numeric': qid})
print(qdoc['_id_numeric'], qdoc['_id'], qdoc['context'])
print(tabulate.tabulate(qdoc['content']))

189421 252.31144 ['2008 K2 disaster', 'List of fatalities']
----------------------------  -----------  --------------------  -------------------------------------------------------------------------------
Name                          Nationality  Location of death     Cause of death
Dren Mandić                   Serbia       Below the Bottleneck  Fell during the ascent
Jehan Baig                    Pakistan     Below the Bottleneck  Fell while trying to recover Dren Mandić's corpse
Rolf Bae                      Norway       Bottleneck            The first serac fall
{{nowrap|Hugues D'Aubarede}}  France       Above the Bottleneck  Fell in descent during the night
Karim Meherban                Pakistan     Above the Bottleneck  Either the second serac fall or the third serac fall
Gerard McDonnell              Ireland      Above the Bottleneck  Hit by either the second or third serac fall, after helping the injured Koreans
Kyeong-Hyo Park               South Korea  Above the Bottleneck  

In [77]:
j = 0

rid = results[j][0]
rdoc = collection.find_one({'_id_numeric': rid})
print(rdoc['_id_numeric'], rdoc['_id'], rdoc['context'])
print(tabulate.tabulate(rdoc['content']))

125546 288.8009 ['List of modern armament manufacturers', '']
----------------------------------------------------------------------------------------------------------------------------------------------------  --------------------------------
Company                                                                                                                                               Country
Établissement de Constructions et de Réparation Navale (ECRN) (:fr:Établissement de construction et de réparation navale)                             Algeria
Bersa S.A. (Bersa)                                                                                                                                    Argentina
Complejo Industrial Naval Argentino (CINAR)                                                                                                           Argentina
Fabricaciones Militares Sociedad del Estado (Fabricaciones Militares)                                                  

In [81]:
metrics = []
results, metrics = sloth(qdoc['content'], rdoc['content'], verbose=True, complete=False, metrics=metrics)
print('Number of distinct overlaps: ', len(results))
for r in results:
    print(tabulate.tabulate([[x] for x in r[0]]))
metrics

Detected 114 seed(s) in 0.024941205978393555 seconds.
'Timed Out'
Generated 16485 candidate(s) in 0.02595806121826172 seconds.
Verified 13792 candidate(s) in 0.24229001998901367 seconds.
Detected 1 largest overlap(s) in 0.2682516574859619 seconds.
Size of the detected largest overlap(s): 9 columns, 1 rows, 9 cells.
Total elapsed time: 6.223509788513184 seconds.
        0         1       2       3         4        5            6  \
0  Serbia  Pakistan  Norway  France  Pakistan  Ireland  South Korea   

             7            8  
0  South Korea  South Korea  
Number of distinct overlaps:  1
--------
(4, 176)
(6, 269)
(3, 388)
(5, 393)
(2, 394)
(1, 457)
(9, 486)
(8, 487)
(7, 499)
--------


[114,
 0.024941205978393555,
 'a',
 32,
 0.0,
 16485,
 0.02595806121826172,
 13792,
 0.24229001998901367,
 1,
 9,
 1,
 9,
 6.223509788513184]

In [82]:
metrics

[114,
 0.024941205978393555,
 'a',
 32,
 0.0,
 16485,
 0.02595806121826172,
 13792,
 0.24229001998901367,
 1,
 9,
 1,
 9,
 6.223509788513184]

In [30]:
qdoc['content'][8]

['7',
 'File:Avilés1.jpg',
 'Gabriel de Avilés,\n2nd Marquis of Avilés',
 '14 March 1799 &ndash; 20 May 1801',
 '25 October 1797',
 '']

In [31]:
rdoc['content'][58]

['25 October 1797',
 'Leicestershire',
 'u*',
 'Penn Assheton Curzon',
 'George Anthony Legh Keck',
 'Death']

In [26]:
results

([([(8, 58)], [('25 October 1797',)])],
 [1,
  0.012362480163574219,
  'e',
  None,
  4.5299530029296875e-06,
  0,
  0.0,
  0,
  0.0,
  1,
  1,
  1,
  1,
  0.012433290481567383])

Examples:

--> same topic, 2012 European Athletics Championships - Women's 4x100 metres relay, results FINAL vs ROUND-1
query 766881, result 766450

## Inspect cities-by-elevation

In [5]:
mongoclient = pymongo.MongoClient()
collection = mongoclient.sloth.demo

In [6]:
docs = list(collection.find({}))
len(docs)
docs = docs[:10]

In [7]:
pairs = []

for i, idoc in enumerate(docs[:-1]):
    print(f'{i}/{len(docs)}', end='\r')
    for j, jdoc in enumerate(docs[i+1:]):
        metrics = []
        results = sloth(parse_table(idoc['content'], idoc['num_columns'], idoc['num_header_rows']), 
                        parse_table(jdoc['content'], jdoc['num_columns'], jdoc['num_header_rows']), 
                        verbose=False, complete=False, metrics=metrics)
        pairs.append((idoc['_id'], jdoc['_id'], metrics[-2]))

In [9]:
pairs

[('358.73464', '358.73465', 1392),
 ('358.73464', '358.73466', 1386),
 ('358.73464', '358.73467', 1386),
 ('358.73464', '358.73468', 1356),
 ('358.73464', '358.73469', 1356),
 ('358.73464', '358.73470', 1350),
 ('358.73464', '358.73471', 1344),
 ('358.73464', '358.73472', 1338),
 ('358.73464', '358.73473', 1338),
 ('358.73465', '358.73466', 1386),
 ('358.73465', '358.73467', 1386),
 ('358.73465', '358.73468', 1356),
 ('358.73465', '358.73469', 1356),
 ('358.73465', '358.73470', 1350),
 ('358.73465', '358.73471', 1344),
 ('358.73465', '358.73472', 1338),
 ('358.73465', '358.73473', 1338),
 ('358.73466', '358.73467', 1392),
 ('358.73466', '358.73468', 1362),
 ('358.73466', '358.73469', 1362),
 ('358.73466', '358.73470', 1356),
 ('358.73466', '358.73471', 1350),
 ('358.73466', '358.73472', 1344),
 ('358.73466', '358.73473', 1338),
 ('358.73467', '358.73468', 1368),
 ('358.73467', '358.73469', 1368),
 ('358.73467', '358.73470', 1362),
 ('358.73467', '358.73471', 1356),
 ('358.73467', '358.

In [10]:
i, j, o = pairs[0]
idoc = collection.find_one({'_id': i})
jdoc = collection.find_one({'_id': j})

results, metrics = sloth(parse_table(idoc['content'], idoc['num_columns'], idoc['num_header_rows']), 
                         parse_table(jdoc['content'], jdoc['num_columns'], jdoc['num_header_rows']), 
                         verbose=True, complete=False)

Detected 8 seed(s) in 0.006017446517944336 seconds.
Generated 1 candidate(s) in 2.4557113647460938e-05 seconds.
Verified 1 candidate(s) in 0.0008642673492431641 seconds.
Detected 1 largest overlap(s) in 0.0009086132049560547 seconds.
Size of the detected largest overlap(s): 6 columns, 232 rows, 1392 cells.
Total elapsed time: 0.007126331329345703 seconds.
                          0                       1  ...          4     5
0               Afghanistan           Kabol / Kabul  ...   E69.1952  1807
1                   Albania         Tirane / Tirana  ...   E19.8172   104
2                   Algeria          Alger /Algiers  ...    E3.0597     0
3       American Samoa (US)  Pago Pago (on Tutuila)  ...  W170.7009    49
4                   Andorra        Andorra la Vella  ...    E1.5218  1409
..                      ...                     ...  ...        ...   ...
227  Wallis and Futuna (FR)    Mata-Utu (on Wallis)  ...  W176.1430     0
228     Western Sahara (MA)     Laayoune / El Aaiu

In [11]:
i, j, o = pairs[1]
idoc = collection.find_one({'_id': i})
jdoc = collection.find_one({'_id': j})

results, metrics = sloth(parse_table(idoc['content'], idoc['num_columns'], idoc['num_header_rows']), 
                         parse_table(jdoc['content'], jdoc['num_columns'], jdoc['num_header_rows']), 
                         verbose=True, complete=False)

Detected 8 seed(s) in 0.005075693130493164 seconds.
Generated 1 candidate(s) in 2.5033950805664062e-05 seconds.
Verified 1 candidate(s) in 0.0008070468902587891 seconds.
Detected 1 largest overlap(s) in 0.0008516311645507812 seconds.
Size of the detected largest overlap(s): 6 columns, 231 rows, 1386 cells.
Total elapsed time: 0.006217241287231445 seconds.
                          0                       1  ...          4     5
0               Afghanistan           Kabol / Kabul  ...   E69.1952  1807
1                   Albania         Tirane / Tirana  ...   E19.8172   104
2                   Algeria          Alger /Algiers  ...    E3.0597     0
3       American Samoa (US)  Pago Pago (on Tutuila)  ...  W170.7009    49
4                   Andorra        Andorra la Vella  ...    E1.5218  1409
..                      ...                     ...  ...        ...   ...
226  Wallis and Futuna (FR)    Mata-Utu (on Wallis)  ...  W176.1430     0
227     Western Sahara (MA)     Laayoune / El Aaiu

In [12]:
results

[([(0, 0), (2, 2), (3, 3), (4, 4), (1, 1), (5, 5)],
  [('Albania', 'Tirane / Tirana', 'Europe', 'N41.3317', 'E19.8172', '104'),
   ('Andorra', 'Andorra la Vella', 'Europe', 'N42.5075', 'E1.5218', '1409'),
   ('Austria', 'Vienna', 'Europe', 'N48.2092', 'E16.3728', '170'),
   ('Belarus', 'Minsk', 'Europe', 'N53.9678', 'E27.5766', '198'),
   ('Belgium', 'Bruxelles / Brussels', 'Europe', 'N50.8371', 'E4.3676', '76'),
   ('Bosnia and Herzegovina',
    'Sarajevo',
    'Europe',
    'N43.8608',
    'E18.4214',
    '577'),
   ('Bulgaria', 'Sofiya / Sofia', 'Europe', 'N42.7105', 'E23.3238', '591'),
   ('Croatia', 'Zagreb', 'Europe', 'N45.8150', 'E15.9785', '130'),
   ('Czech Republic',
    'Praha / Prague',
    'Europe',
    'N50.0878',
    'E14.4205',
    '244'),
   ('Denmark',
    'Kobenhavn / Copenhagen',
    'Europe',
    'N55.6763',
    'E12.5681',
    '0'),
   ('Estonia', 'Tallinn', 'Europe', 'N59.4389', 'E24.7545', '37'),
   ('Finland', 'Helsinki', 'Europe', 'N60.1699', 'E24.9384', '25')